# TicTacToe

Vi skal nå se på hvordan vi kan bruke TD læring til å trene en agent til å spille TicTacToe

In [ ]:
import os
import sys
sys.path.append(os.getcwd())
%pip install matplotlib numpy

In [ ]:
%matplotlib inline
import numpy as np
import pickle
from State import all_states
from TicTacEnv import TicTacEnv
import matplotlib.pyplot as plt

Tic Tac Toe er ferdig implementert, men vi har lyst på mulighet til å spille selv

In [ ]:
class HumanAgent:

    def __init__(self, **kwargs):
        self.symbol = None
        self.keys = ['q', 'w', 'e', 'a', 's', 'd', 'z', 'x', 'c']
        self.state = None

    def reset(self):
        pass

    def set_state(self, state):
        self.state = state

    def set_symbol(self, symbol):
        self.symbol = symbol

    def act(self):
        self.state.render()
        key = input("Input your position:")
        data = self.keys.index(key)
        i = data // 3
        j = data % 3
        return i, j, self.symbol

Det er ofte også nyttig med mulighet for tilfeldige handlinger

In [ ]:
def get_random_move(state, symbol):
    next_states = []
    next_positions = []
    for i in range(3):
        for j in range(3):
            if state.board[i, j] == 0:
                next_positions.append([i, j])
                next_states.append(state.next_state(i, j, symbol).hash())
    
    action = next_positions[np.random.randint(len(next_positions))]
    action.append(symbol)
    return action
    

Vi lager en agent som kan spille tilfeldig

In [ ]:
class RandomAgent:
    def __init__(self, **kwargs):
        self.symbol = None
        self.state = None
        
    def reset(self):
        pass

    def set_state(self, state):
        self.state = state

    def set_symbol(self, symbol):
        self.symbol = symbol

    def act(self):
        return get_random_move(self.state, self.symbol)

Og mulighet til å spille mot tifleldig agenten som spiller tilfeldig

In [ ]:
def play():
    for i in range(2):
        # Load agents
        player1 = TODO
        player2 = TODO
        # Load environment
        env = TicTacEnv(player1, player2)
        # Play the game
        winner = env.play()
        if winner == player2.symbol:
            print("You lose!")
        elif winner == player1.symbol:
            print("You win!")
        else:
            print("It is a tie!")

Show the random agent what you are made of!

In [ ]:
play()

Lets play the random agent against itself

In [ ]:
def compete(turns):
    player1 = RandomAgent()
    player2 = RandomAgent()
    env = TicTacEnv(player1, player2)
    player1_win = 0.0
    player2_win = 0.0
    tie = 0.0
    for _ in range(turns):
        winner = env.play()
        if winner == 1:
            player1_win += 1
        elif winner == -1:
            player2_win += 1
        else:
            tie += 1
        env.reset()
    print('%d turns, player 1 win %.02f, player 2 win %.02f, tied %.02f'% (turns, player1_win / turns, player2_win / turns, tie / turns))

In [ ]:
compete(100)

How did it do? Was it what you would expect?
What if you change the number of games?

## RL agent

This is where most of your code will go.

Fill inn all the TODOs
Feel free to ask questions

In [ ]:
class Agent:

    def __init__(self, learning_rate=0.1, epsilon=0.1):
        self.value_table = dict()
        self.learning_rate = TODO
        self.epsilon = TODO
        self.states = []
        self.greedy = []
        self.symbol = 0

    def reset(self):
        # Reset the agents so its ready for a new episode
        self.states = []
        self.greedy = []

    def set_state(self, state):
        self.states.append(state)
        self.greedy.append(True)

    def set_symbol(self, symbol):
        # Set the symbol the agent will use
        self.symbol = symbol
        for hash_val in all_states:
            state, is_end = all_states[hash_val]
            if is_end:
                if state.winner == self.symbol:
                    self.value_table[hash_val] = 1.0
                elif state.winner == 0:
                    # we need to distinguish between a tie and a lose
                    self.value_table[hash_val] = 0.5
                else:
                    self.value_table[hash_val] = 0
            else:
                self.value_table[hash_val] = 0.5

    def backup(self):
        # Back up the values we have encountered
        states = [state.hash() for state in self.states]

        for i in reversed(range(len(states) - 1)):
            # For each state we want to compute the TD error or "Surprise" of the system
            state = TODO
            td_error = TODO
            self.value_table[state] += TODO

    def act(self):
        # Selecting an action for the agent to performe
        state = self.states[-1]
        next_states = []
        next_positions = []
        for i in range(3):
            for j in range(3):
                if state.board[i, j] == 0:
                    next_positions.append([i, j])
                    next_states.append(state.next_state(i, j, self.symbol).hash())

        # If we have random less than epsilon we select an action randomly
        if TODO:
            action = next_positions[np.random.randint(len(next_positions))]
            action.append(self.symbol)
            self.greedy[-1] = False
            return action

        # Find our values for the next states
        values = []
        for hash_val, pos in zip(next_states, next_positions):
            values.append((self.value_table[hash_val], pos))

        # Lets sort all next states in decreasing value order
        np.random.shuffle(values)
        values.sort(key=lambda x: x[0], reverse=True)
        action = values[0][1]
        action.append(self.symbol)
        # Return action that leads to the next state with highest value
        return action

    # It is very usefull to save our policy when we are done training
    def save_policy(self):
        with open('policy_%s.bin' % ('first' if self.symbol == 1 else 'second'), 'wb') as f:
            pickle.dump(self.value_table, f)

    # Likewise it is nice to be able to load our policy again
    def load_policy(self):
        with open('policy_%s.bin' % ('first' if self.symbol == 1 else 'second'), 'rb') as f:
            self.value_table = pickle.load(f)

Now that we have the agent, we need to train it to play the game
Again fill in the TODOs

In [ ]:
class Trainer:

    @staticmethod
    def train(epochs, print_interval=500):
        # Initialize the agents
        p1 = TODO
        p2 = TODO

        # Initialize the environment
        env = TODO
        p1_wins = 0
        p2_wins = 0

        p1_100 = 0
        p2_100 = 0
        p1_history = []
        p2_history = []
        tie_history = []

        # for each episode
        for i in range(1, epochs + 1):
            # Play a round and append the results
            winner = TODO
            if winner == 1:
                p1_wins += 1
                p1_100 += 1
            if winner == -1:
                p2_wins += 1
                p2_100 += 1
            # Print ocationaly
            if i % print_interval == 0:
                print('Epoch %d, player 1 winrate: %.02f, player 2 winrate: %.02f' % (
                    i, p1_wins / i, p2_wins / i))
            if i % 250 == 0:
                p1_history.append(p1_100 / 250)
                p2_history.append(p2_100 / 250)
                tie_history.append((250 - p1_100 - p2_100) / 250)
                p1_100 = 0
                p2_100 = 0

            # Train our agents
            TODO
            TODO

            # Reset the envoronment
            TODO

        # Save the policies
        TODO
        TODO

        # Plot the training hisory
        plt.plot(p1_history, label="p1")
        plt.plot(p2_history, label="p2")
        plt.plot(tie_history, label="tie")
        plt.legend()
        plt.show()

In [ ]:
Trainer.train(TODO)

Lets test the two agents agains eachother

In [ ]:
def compete(turns):
    player1 = TODO
    player2 = TODO
    env = TicTacEnv(player1, player2)
    player1.load_policy()
    player2.load_policy()
    player1_win = 0.0
    player2_win = 0.0
    for _ in range(turns):
        winner = env.play()
        if winner == 1:
            player1_win += 1
        if winner == -1:
            player2_win += 1
        env.reset()
    print('%d turns, player 1 win %.02f, player 2 win %.02f' % (turns, player1_win / turns, player2_win / turns))


In [ ]:
compete(int(1e3))

How did it go? Did it play as expected?

Play against it yourself!

In [ ]:
def play():
    for i in range(2):
        player1 = HumanAgent()
        player2 = Agent(epsilon=0)
        env = TicTacEnv(player1, player2)
        player2.load_policy()
        winner = env.play()
        if winner == player2.symbol:
            print("You lose!")
        elif winner == player1.symbol:
            print("You win!")
        else:
            print("It is a tie!")

In [ ]:
play()

Done?

Try to train if for longer, shorter.